In [4]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split


In [9]:
df = pd.read_csv('alldata.csv')

In [10]:
selected_columns = ['CGM_Data', 'carbs', 'calories', 'fiber', 'fat', 'protein', 
                    'Gender (Female=1, Male=2)', 'Age (years)', 'BMI (kg/m2)', 
                    'Weight (kg)', 'Hour', 'Duration of diabetes (years)','avgCGM']
df_selected = df[selected_columns]

In [12]:
train_df, test_df = train_test_split(df_selected, test_size=0.2)


In [13]:
train_labels = train_df.pop('avgCGM')
test_labels = test_df.pop('avgCGM')
train_features = dict(train_df)
test_features = dict(test_df)

In [14]:
def input_fn(features, labels, training=True, batch_size=32):
    selected_columns = ['CGM_Data', 'carbs', 'calories', 'fiber', 'fat', 'protein', 
                        'Gender (Female=1, Male=2)', 'Age (years)', 'BMI (kg/m2)', 
                        'Weight (kg)', 'Hour', 'Duration of diabetes (years)']
    selected_features = {k: features[k] for k in selected_columns}
    dataset = tf.data.Dataset.from_tensor_slices((selected_features, labels))
    if training:
        dataset = dataset.shuffle(1000).repeat()
    return dataset.batch(batch_size)

In [15]:
feature_columns = []
for feature_name in selected_columns[:-1]:  # exclude the label column
    if df[feature_name].dtype == object:
        vocabulary = df[feature_name].unique()
        feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))
    else:
        feature_columns.append(tf.feature_column.numeric_column(feature_name))


In [16]:
estimator = tf.estimator.BoostedTreesRegressor(feature_columns=feature_columns, n_batches_per_layer=1)

AttributeError: module 'tensorflow_estimator.python.estimator.api._v2.estimator' has no attribute 'BoostedTreesRegressor'

In [ ]:
estimator.train(input_fn=lambda: input_fn(train_features, train_labels, True, 32), steps=1000)

In [ ]:
eval_result = estimator.evaluate(input_fn=lambda: input_fn(test_features, test_labels, False, 32))

In [ ]:
print(eval_result)

In [17]:
print(tf.__version__)

2.11.0
